# Import libraries

In [10]:
import subprocess
from pathlib import Path
import pycolmap
import os
import shutil


# Functions

In [11]:
def clean_colmap(base_dir="assets"):
    """
    Recursively delete all COLMAP-generated files such as `pycolmap.db` 
    and `pycolmap_output/` directories from the given base directory.

    Parameters:
        base_dir (str): Root directory to start cleanup (default: 'assets')
    """
    for root, dirs, files in os.walk(base_dir):
        # Delete pycolmap.db files
        if "pycolmap.db" in files:
            db_path = os.path.join(root, "pycolmap.db")
            os.remove(db_path)
            print(f"Deleted file: {db_path}")

        # Delete pycolmap_output directories
        if "pycolmap_output" in dirs:
            output_path = os.path.join(root, "pycolmap_output")
            shutil.rmtree(output_path)
            print(f"Deleted directory: {output_path}")

In [8]:
def prepare_database_with_colmap(image_dir):
    image_dir = Path(image_dir)
    db_path = image_dir / "pycolmap.db"
    
    if db_path.exists():
        print("ℹ️ Removing existing database to force fresh extraction and matching.")
        db_path.unlink()
    # 1. Feature extraction
    subprocess.run([
        "colmap", "feature_extractor",
        "--database_path", str(db_path),
        "--image_path", str(image_dir)
    ], check=True)

    # 2. Feature matching
    subprocess.run([
        "colmap", "exhaustive_matcher",
        "--database_path", str(db_path)
    ], check=True)

    return db_path

def run_pycolmap_sfm_after_cli(image_dir):
    image_dir = Path(image_dir)
    db_path = prepare_database_with_colmap(image_dir)
    output_dir = image_dir / "pycolmap_output"
    output_dir.mkdir(exist_ok=True)

    reconstructions = pycolmap.incremental_mapping(
        database_path=str(db_path),
        image_path=str(image_dir),
        output_path=str(output_dir)
    )

    if not reconstructions:
        raise RuntimeError("❌ No reconstruction produced.")
    
    sfm = reconstructions[0]
    print("✅ SfM complete")
    print(f"Images registered: {len(sfm.images)}")
    print(f"3D points: {len(sfm.points3D)}")
    return sfm

In [12]:
clean_colmap()

Deleted file: assets/marine_noisy/pycolmap.db
Deleted directory: assets/marine_noisy/pycolmap_output
Deleted file: assets/marine/pycolmap.db
Deleted directory: assets/marine/pycolmap_output


In [16]:
image_dir = "assets/marine"
db_path = f"{image_dir}/pycolmap.db"
subprocess.run([
    "colmap", "feature_extractor",
    "--database_path", str(db_path),
    "--image_path", str(image_dir)
], check=True)

QStandardPaths: wrong permissions on runtime directory /run/user/1000/, 0755 instead of 0700
I20250715 18:12:51.563297 39293 misc.cc:198] 
Feature extraction
I20250715 18:12:52.621860 39323 feature_extraction.cc:254] Processed file [1/22]
I20250715 18:12:52.621906 39323 feature_extraction.cc:257]   Name:            Screenshot 2025-07-15 123438.png
I20250715 18:12:52.621912 39323 feature_extraction.cc:283]   Dimensions:      2560 x 1600
I20250715 18:12:52.621917 39323 feature_extraction.cc:286]   Camera:          #1 - SIMPLE_RADIAL
I20250715 18:12:52.621938 39323 feature_extraction.cc:289]   Focal Length:    3072.00px
I20250715 18:12:52.621953 39323 feature_extraction.cc:302]   Features:        5403
I20250715 18:12:53.128152 39323 feature_extraction.cc:254] Processed file [2/22]
I20250715 18:12:53.128202 39323 feature_extraction.cc:257]   Name:            Screenshot 2025-07-15 123455.png
I20250715 18:12:53.128208 39323 feature_extraction.cc:283]   Dimensions:      2560 x 1600
I20250715 

CompletedProcess(args=['colmap', 'feature_extractor', '--database_path', 'assets/marine/pycolmap.db', '--image_path', 'assets/marine'], returncode=0)

In [9]:
sfm_result = run_pycolmap_sfm_after_cli("assets/marine")

ℹ️ Removing existing database to force fresh extraction and matching.


QStandardPaths: wrong permissions on runtime directory /run/user/1000/, 0755 instead of 0700
I20250715 17:11:23.194550 38229 misc.cc:198] 
Feature extraction
I20250715 17:11:24.067042 38259 feature_extraction.cc:254] Processed file [1/22]
I20250715 17:11:24.067085 38259 feature_extraction.cc:257]   Name:            Screenshot 2025-07-15 123438.png
I20250715 17:11:24.067091 38259 feature_extraction.cc:283]   Dimensions:      2560 x 1600
I20250715 17:11:24.067096 38259 feature_extraction.cc:286]   Camera:          #1 - SIMPLE_RADIAL
I20250715 17:11:24.067101 38259 feature_extraction.cc:289]   Focal Length:    3072.00px
I20250715 17:11:24.067116 38259 feature_extraction.cc:302]   Features:        5403
I20250715 17:11:24.404017 38259 feature_extraction.cc:254] Processed file [2/22]
I20250715 17:11:24.404062 38259 feature_extraction.cc:257]   Name:            Screenshot 2025-07-15 123455.png
I20250715 17:11:24.404068 38259 feature_extraction.cc:283]   Dimensions:      2560 x 1600
I20250715 

RuntimeError: ❌ No reconstruction produced.

In [6]:
sfm_result.visualize()

AttributeError: 'pycolmap._core.Reconstruction' object has no attribute 'visualize'